In [2]:
!pip install torch torchvision efficientnet_pytorch pandas scikit-learn

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16427 sha256=5549c08e8f240f1d88f5ce38579639a18153a6acf44fbe2158f79dde7623cb64
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch


In [15]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from efficientnet_pytorch import EfficientNet
from torchvision import transforms
import matplotlib.pyplot as plt
from torch import nn, optim
from tqdm import tqdm
from PIL import Image
import pandas as pd
import torch
import copy
import os

In [16]:
# Set device (use GPU ONLY)

device = torch.device('cuda')

In [17]:
class DiabeticRetinopathyDataset(Dataset):
    def __init__(self, csv_file, image_dir, transform=None, is_test=False):
        self.data = pd.read_csv(csv_file)
        self.image_dir = image_dir  # Image directory
        self.transform = transform
        self.is_test = is_test
        self.skip_images = {'194_right.jpeg', '29126_right.jpeg', '38790_right.jpeg', '8421_left.jpeg'}  # Images to skip

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_name = self.data.iloc[idx]['image']

        # Skip specified images and return a dummy tensor and a default label
        if image_name in self.skip_images:
            print(f"Skipping image {image_name} as per request")
            dummy_image = torch.zeros(3, 224, 224)  # A dummy image tensor (3x224x224)
            return dummy_image, 0  # Returning dummy tensor and label '0'

        image_path = os.path.join(self.image_dir, image_name)  # Path to the image
        try:
            image = Image.open(image_path).convert('RGB')
        except (OSError, IOError) as e:
            print(f"Error loading image {image_name}, skipping. Error: {e}")
            dummy_image = torch.zeros(3, 224, 224)
            return dummy_image, 0

        if self.transform:
            image = self.transform(image)

        if self.is_test:
            return image, image_name
        else:
            label = self.data.iloc[idx]['level']
            return image, label

In [18]:
# Define image transformations

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [19]:
# Load data

train_df = pd.read_csv('/kaggle/input/diabetic-retinopathy/train.csv')
train_data, val_data = train_test_split(train_df, test_size=0.4, random_state=42)
train_df

,image,level
0,10_left.jpeg,0
1,10_right.jpeg,0
2,13_left.jpeg,0
3,13_right.jpeg,0
4,15_left.jpeg,1
...,...,...
33694,44347_right.jpeg,0
33695,44348_left.jpeg,0
33696,44348_right.jpeg,0
33697,44349_left.jpeg,0


In [20]:
# Save train and val splits to temporary CSVs

train_image_dir = "/kaggle/input/diabetic-retinopathy/train/"
test_image_dir = "/kaggle/input/diabetic-retinopathy/test/"

train_data.to_csv('/kaggle/working/train_split.csv', index=False)
val_data.to_csv('/kaggle/working/val_split.csv', index=False)

train_dataset = DiabeticRetinopathyDataset('train_split.csv', image_dir=train_image_dir, transform=train_transform)
val_dataset = DiabeticRetinopathyDataset('val_split.csv', image_dir=train_image_dir, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4)

In [21]:
# Initialize EfficientNet model

model = EfficientNet.from_pretrained('efficientnet-b0')
num_ftrs = model._fc.in_features
model._fc = nn.Linear(num_ftrs, 5)  # 5 output classes (0-4 for diabetic retinopathy)

model = model.to(device)

Loaded pretrained weights for efficientnet-b0


In [22]:
# Loss function and optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)  # Reduce LR every 5 epochs

In [23]:
# Function to evaluate the model on validation set

def evaluate(model, val_loader, criterion):
    
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        
        for images, labels in tqdm(val_loader, desc="Validation"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    
    val_accuracy = correct / total
    val_loss /= len(val_loader)
    
    return val_loss, val_accuracy

In [24]:
# Function to train the model

def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=20):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_accuracy = 0.0
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        
        # Training phase
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for images, labels in tqdm(train_loader, desc="Training"):
            # Skip if image or label is None (invalid/truncated image)
            if images is None or labels is None:
                continue

            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = correct / total
        print(f'Training Loss: {epoch_loss:.4f}, Training Accuracy: {epoch_acc:.4f}')

        # Validation phase
        val_loss, val_accuracy = evaluate(model, val_loader, criterion)
        print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

        # Save the best model
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(model.state_dict(), f'/kaggle/working/best_model_epoch_{epoch+1}.pth')
            print(f'Saved best model for epoch {epoch+1} at /kaggle/working/')

        # Step the learning rate scheduler
        scheduler.step()

    model.load_state_dict(best_model_wts)
    return model

In [25]:
# Train the model

model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=20)

Epoch 1/20


Training:  17%|█▋        | 213/1264 [02:43<09:22,  1.87it/s]

Skipping image 29126_right.jpeg as per request


Training:  36%|███▌      | 451/1264 [05:44<11:52,  1.14it/s]

Skipping image 8421_left.jpeg as per request


...

Skipping image 194_right.jpeg as per request


Training: 100%|██████████| 1264/1264 [16:21<00:00,  1.29it/s]


Training Loss: 0.4695, Training Accuracy: 0.8420


Validation: 100%|██████████| 843/843 [10:47<00:00,  1.30it/s]

Validation Loss: 0.5658, Validation Accuracy: 0.8168


In [26]:
# Test predictions

model_path = "/kaggle/working/best_model_epoch_14.pth"
model.load_state_dict(torch.load(model_path))  # Load the state_dict from the saved model
model.to(device)

test_dataset = DiabeticRetinopathyDataset('/kaggle/input/diabetic-retinopathy/test.csv', image_dir=test_image_dir, transform=val_transform, is_test=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)

model.eval()
predictions = []

with torch.no_grad():
    for images, image_paths in tqdm(test_loader, desc="Testing"):
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        predictions.extend(list(zip(image_paths, preds.cpu().numpy())))

/tmp/ipykernel_31/2401850270.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))  # Load the state_dict from the saved model
Te

Error loading image 1626_left.jpeg, skipping. Error: image file is truncated (38 bytes not processed)


Testing: 100%|█████████▉| 1051/1055 [13:39<00:03,  1.30it/s]

Error loading image 22554_right.jpeg, skipping. Error: image file is truncated (47 bytes not processed)


Testing: 100%|██████████| 1055/1055 [13:40<00:00,  1.29it/s]


In [27]:
# Save predictions to CSV

pred_df = pd.DataFrame(predictions, columns=['image', 'level'])
pred_df.to_csv('/kaggle/working/predictions.csv', index=False)
print("saved to predictions.csv")

saved to predictions.csv


In [70]:
image1 = Image.open('/kaggle/input/diabetic-retinopathy/train/10007_right.jpeg').convert('RGB')

In [28]:
asfasdf = pd.read_csv('/kaggle/working/predictions.csv')
asfasdf

,image,level
0,10000_left.jpeg,0
1,10000_right.jpeg,0
2,10001_left.jpeg,2
3,10001_right.jpeg,2
4,10002_left.jpeg,0
...,...,...
16867,22552_right.jpeg,2
16868,22553_left.jpeg,0
16869,22553_right.jpeg,0
16870,22554_left.jpeg,0
